In [7]:
# Install Selenium and WebDriver Manager
!pip install selenium
!pip install webdriver-manager
!pip install tqdm
!pip install pytesseract

# Install Chrome
!apt-get update
!apt-get install -y wget unzip
!wget -q https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
!dpkg -i google-chrome-stable_current_amd64.deb
!apt-get -f install -y

# Check the versions of Chrome
!google-chrome --version


^C
ERROR: Operation cancelled by user
zsh:1: command not found: apt-get
zsh:1: command not found: apt-get
zsh:1: command not found: wget
zsh:1: command not found: dpkg
zsh:1: command not found: apt-get
zsh:1: command not found: google-chrome


In [ ]:
# MacOS Version
# Install Selenium and WebDriver Manager
!pip install selenium
!pip install webdriver-manager
!pip install tqdm

# Install Homebrew if not installed
!which brew || /bin/bash -c "$(curl -fsSL https://raw.githubusercontent.com/Homebrew/install/HEAD/install.sh)"

# Install Chrome using Homebrew
!brew install --cask google-chrome

# Check the version of Chrome
!"/Applications/Google Chrome.app/Contents/MacOS/Google Chrome" --version


/opt/homebrew/bin/brew
==> Auto-updating Homebrew...
Adjust how often this is run with HOMEBREW_AUTO_UPDATE_SECS or disable with
HOMEBREW_NO_AUTO_UPDATE. Hide these hints with HOMEBREW_NO_ENV_HINTS (see `man brew`).
==> Downloading https://dl.google.com/chrome/mac/universal/stable/GGRO/googlechr
######################################################################### 100.0%
==> Installing Cask google-chrome
==> Purging files for version 125.0.6422.113 of Cask google-chrome
Error: It seems there is already an App at '/Applications/Google Chrome.app'.
Google Chrome 125.0.6422.78 


In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
from datetime import datetime, timedelta
from bs4 import BeautifulSoup
from time import sleep
import pandas as pd
import requests
from tqdm import tqdm
import random
import os



In [2]:
# Lib for image recognition
import pytesseract
from PIL import Image
from io import BytesIO


In [3]:
# define driver
def setup_driver():
    chrome_options = webdriver.ChromeOptions()
    # options.add_argument('--headless')
    # options.add_argument('--disable-gpu')
    # options.add_argument('--no-sandbox')
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')
    chrome_options.add_argument('--disable-gpu')
    chrome_options.add_argument('--window-size=1920x1080')
    chrome_options.add_argument("--disable-blink-features=AutomationControlled")
    chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
    chrome_options.add_experimental_option('useAutomationExtension', False)
    chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3")
    chrome_options.add_argument("accept=text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8")
    chrome_options.add_argument("accept-language=en-US,en;q=0.9")
    chrome_options.add_argument("accept-encoding=gzip, deflate, br")
    chrome_options.add_argument("upgrade-insecure-requests=1")
    chrome_options.add_argument("cache-control=no-cache")
    # driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
    driver = webdriver.Chrome(options=chrome_options) # if error, use this version
    return driver



In [4]:
# Function to perform random sleep to mimic human behavior
def random_sleep(min_seconds, max_seconds):
    sleep(random.uniform(min_seconds, max_seconds))

def debug_page(filename_prefix, filings='All'):
    directory = f'debug_dir/{filings}'
    if not os.path.exists(directory):
        os.makedirs(directory)
    # page_source = driver.page_source
    # with open(f"{directory}/{filename_prefix}_page_source.html", "w", encoding="utf-8") as file:
    #     file.write(page_source)
    driver.save_screenshot(f"{directory}/{filename_prefix}_screenshot.png")
    print(f"Debug info saved: {filename_prefix}_page_source.html and {filename_prefix}_screenshot.png")

# Function to handle the Terms of Use page
def handle_terms_of_use():
    try:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        accept_button = WebDriverWait(driver, 5).until(
            EC.element_to_be_clickable((By.ID, 'ctl00_mainContentArea_disclaimerContent_yesButton'))
        )
        # debug_page("terms_of_use")
        accept_button.click()
        random_sleep(1, 2)  # Wait a bit to ensure the page loads completely
        # print("Accepted Terms of Use")
        return True
    except Exception as e:
        print(f"No Terms of Use page found or unable to click accept button: ")
        # debug_page("terms_of_use_error")
    return False

## Scraping Search Results

In [5]:
def check_search_result_window():
    try:
        # Wait for the counter label to appear
        search_results_loaded = WebDriverWait(driver, 30).until(
            EC.presence_of_element_located((By.CLASS_NAME, 'searchResultsSecurityView'))
        )
    except Exception as e:
        print("Search results window did not load")
        return False

# Detect large search results
def is_large_search_result():
    try:
        large_search_message = driver.find_element(By.ID, 'counterLabel')
        return "Please use filters to narrow your search." in large_search_message.text
    except Exception as e:
        return False



In [6]:
def select_date(input_id, date):
    month = date.strftime("%b")
    year = date.strftime("%Y")
    day = date.strftime("%d").lstrip("0")

    input_element = driver.find_element(By.ID, input_id)
    if input_element:
        input_element.click()
    # random_sleep(1, 2)
    # debug_page(f"select_date_{input_id}")

    year_selector = Select(driver.find_element(By.CLASS_NAME, 'ui-datepicker-year'))
    if year_selector:
        year_selector.select_by_visible_text(year)
    # debug_page(f"select_date_year_{input_id}")
    month_selector = Select(driver.find_element(By.CLASS_NAME, 'ui-datepicker-month'))
    if month_selector:
        month_selector.select_by_visible_text(month)
    # debug_page(f"select_date_month_{input_id}")
    day_element = driver.find_element(By.XPATH, f"//a[@data-date='{day}']")
    if day_element:
        day_element.click()
    # debug_page(f"select_date_day_{input_id}")

# Function to select the necessary checkboxes
def select_filings(filings):
    # for filing in filings:
    #   try:
    #       label = driver.find_element(By.XPATH, f'//label[text()="{filing}"]')
    #       checkbox = label.find_element(By.XPATH, './preceding-sibling::input[@type="checkbox"]')
    #       # print(label.text)
    #       checkbox.click()
    #       # print(f"Selected filing: {filing}")
    #   except Exception as e:
    #       print(f"Unable to select filing {filing}: {e}")
    #       debug_page(f"select_filing_error_{filing}")
    
    # for one checkbox only
    try:
        label = driver.find_element(By.XPATH, f'//label[text()="{filings}"]')
        checkbox = label.find_element(By.XPATH, './preceding-sibling::input[@type="checkbox"]')
        # print(label.text)
        checkbox.click()
        # print(f"Selected filing: {filing}")
    except Exception as e:
        print(f"Unable to select filing {filings}: {e}")
        debug_page(f"select_filing_error_{filings}")

# Function to perform the search
def perform_search(start_date, end_date, filings):
    
    print('=' * 50)
    print(f"scraping {filings} from {start_date.strftime('%Y-%m-%d')} to {end_date.strftime('%Y-%m-%d')}")
    def click_disclosures_tab():
        try:
            disclosures_tab = WebDriverWait(driver, 5).until(
                EC.element_to_be_clickable((By.ID, 'disclosuresFilterLi'))
            )
            disclosures_tab.click()
            disclosures_tab.click()
            disclosures_tab.click()
            disclosures_tab.click()
            disclosures_tab.click()
            print('clicked on disclosures tab')
            # random_sleep(2, 4)
            # debug_page("disclosures_tab"， filings)
        except Exception as e:
            print(f"Unable to locate or click the Disclosures tab")
            debug_page("disclosures_tab_error", filings)
            
    # Narrow the search by applying additional filters
    def narrow_search_and_scrape():
        try:
            expand_all_filters = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.ID, 'expandAllFilter'))
            )
            expand_all_filters.click()
            expand_all_filters.click()
            expand_all_filters.click()
            expand_all_filters.click()
            
            random_sleep(2, 4)
            
            filter_dropdown = Select(driver.find_element(By.ID, 'securedByDropdownList'))
            options = [option.get_attribute('value') for option in filter_dropdown.options if option.get_attribute('value')]
            df_li = []
            for option in options:
                print(f"Applying filter: {option}")
                filter_dropdown.select_by_value(option)
                random_sleep(2, 4)
                
                # Click the "Run Search" button again
                run_search_button = driver.find_element(By.ID, 'runSearchButton')
                run_search_button.click()
                random_sleep(2, 4)
                
                # Wait for the search results to load
                WebDriverWait(driver, 30).until(
                    EC.presence_of_element_located((By.CLASS_NAME, 'searchResultsSecurityView'))
                )
                print("Window loaded")
                element = WebDriverWait(driver, 120).until(
                    lambda driver: driver.find_element(By.ID, "tabs").get_attribute("style") == "display: block;"
                )
                print(f"Results for filter {option} loaded")
                
                # if check_search_result_window():
                length_selector = Select(WebDriverWait(driver, 60).until(
                EC.presence_of_element_located((By.NAME, 'lvDocuments_length'))
                ))
                length_selector.select_by_value('100')
                # debug_page("length_select")
                counter_label = WebDriverWait(driver, 5).until(
                    EC.presence_of_element_located((By.ID, 'counterLabel'))
                )
                num_disclosures = counter_label.text.split()[0]
                print(f"Number of disclosures: {num_disclosures}")
                if num_disclosures == '0':
                    # print(f"No results found from {start_date.strftime('%Y-%m-%d')} to {end_date.strftime('%Y-%m-%d')}")
                    return None
                
                df = scrape_results(num_disclosures, filings)
                print(f'number of rows: {len(df)}')
                df_li.append(df)
                expand_all_filters = WebDriverWait(driver, 10).until(
                    EC.element_to_be_clickable((By.ID, 'expandAllFilter'))
                )
                
                expand_all_filters.click()
                expand_all_filters.click()
                expand_all_filters.click()
                expand_all_filters.click()  
            
            random_sleep(2, 4)
            df = pd.concat(df_li)
            if not os.path.exists(f'results/{filings.replace("/", "_")}'):
                os.makedirs(f'results/{filings.replace("/", "_")}')
            filename = f"results/{filings.replace("/", "_")}/results_{start_date.strftime('%Y-%m-%d')}_{end_date.strftime('%Y-%m-%d')}.csv"
            df.to_csv(filename, index=False)
            print('*'*10, filename, 'Saved')
            return df

                
            
        except Exception as e:
            print(f"Error during narrow_search_and_scrape: {e}")
            debug_page("narrow_search_error_{start_date.strftime('%Y-%m-%d')}_{end_date.strftime('%Y-%m-%d')", filings)
    try:
        driver.get('https://emma.msrb.org/Search/Search.aspx')
        # Handle the terms of use if it appears
        if len(driver.find_elements(By.CLASS_NAME, 'contentAreaDisclaimer')) > 0:
            # print('terms_of_use found')
            handle_terms_of_use()

        # Click on the Disclosures tab
        try:
            disclosures_tab = WebDriverWait(driver, 5).until(
                EC.element_to_be_clickable((By.ID, 'disclosuresFilterLi'))
            )
            disclosures_tab.click()
            disclosures_tab.click()
            disclosures_tab.click()
            disclosures_tab.click()
            disclosures_tab.click()
            print('clicked on disclosures tab')
            # random_sleep(2, 4)
            # debug_page("disclosures_tab"， filings)
        except Exception as e:
            print(f"Unable to locate or click the Disclosures tab")
            # debug_page("disclosures_tab_error")
            print('try terms_of_use')
            if handle_terms_of_use():
                df = perform_search(start_date, end_date, filings)
                return df
            else:
                print('=' * 50)
                print('error when handling terms of use')
                return
        try: 
            WebDriverWait(driver, 10).until(
                EC.visibility_of_element_located((By.XPATH, "//h4[text()='Disclosures']"))
            )
        except Exception as e:
            print('Disclosures tab not loaded')
            click_disclosures_tab()
        
        # Input the start and end dates

        select_date('postingDateFrom', start_date)
        select_date('postingDateTo', end_date)

        # Select the necessary checkboxes
        select_filings(filings)

        # debug_page("checkbox")
        # Click the "Run Search" button
        run_search_button = driver.find_element(By.ID, 'runSearchButton')
        run_search_button.click()
        random_sleep(2, 4)
        
        print('*** page loaded')
        # debug_page("run_search")
        if not check_search_result_window():
            if is_large_search_result():
                print("Large search result detected")
                df = narrow_search_and_scrape()
                return df
        length_selector = Select(WebDriverWait(driver, 60).until(
        EC.presence_of_element_located((By.NAME, 'lvDocuments_length'))
        ))
        length_selector.select_by_value('100')
        # debug_page("length_select")
        counter_label = WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((By.ID, 'counterLabel'))
        )
        num_disclosures = counter_label.text.split()[0]
        print(f"Number of disclosures: {num_disclosures}")
        if num_disclosures == '0':
            # print(f"No results found from {start_date.strftime('%Y-%m-%d')} to {end_date.strftime('%Y-%m-%d')}")
            return None
        df = scrape_results(num_disclosures, filings)
        if not os.path.exists(f'results/{filings}'):
            os.makedirs(f'results/{filings}')
        filename = f"results/{filings}/results_{start_date.strftime('%Y-%m-%d')}_{end_date.strftime('%Y-%m-%d')}.csv"
        df.to_csv(filename, index=False)
        print('*'*10, filename, 'Saved')
        
        return df

    except Exception as e:
        print('*'*10, f"Error during perform_search from {start_date.strftime('%Y-%m-%d')} to {end_date.strftime('%Y-%m-%d')}")
        debug_page(f"perform_search_error_{start_date.strftime('%Y-%m-%d')}_{end_date.strftime('%Y-%m-%d')}", filings)
        return None


In [7]:
# Function to scrape the search results
def scrape_results(num_disclosures, filings='All'):
    results = []
    page = 1
    try:
        pbar = tqdm(total=int(num_disclosures), desc='Scraping results', unit='rows')
        while True:
            print('scraping page: ', page)
            WebDriverWait(driver, 5).until(
                EC.visibility_of_element_located((By.ID, 'lvDocuments_wrapper'))
            )
            # debug_page("scrape_results")

            # Scrape the search results from the current page
            rows = driver.find_elements(By.CSS_SELECTOR, '#lvDocuments tbody tr')
            # print('Number of rows: ', len(rows))
            for row in rows:
                issuer_name = row.find_element(By.CSS_SELECTOR, 'td:nth-child(1)').text
                disclosure_desc = row.find_element(By.CSS_SELECTOR, 'td:nth-child(2) a').text
                link = row.find_element(By.CSS_SELECTOR, 'td:nth-child(2) a').get_attribute('href')
                date_posted = row.find_element(By.CSS_SELECTOR, 'td:nth-child(3)').text
                results.append({
                    'issuer_name': issuer_name,
                    'disclosure_desc': disclosure_desc,
                    'link': link,
                    'date_posted': date_posted
                })
                # print(f"Issuer: {issuer_name}, Description: {disclosure_desc}, Link: {link}, Date: {date_posted}")

            pbar.update(len(rows))
            # print(f"{(page - 1)* 100 + len(rows)} / {num_disclosures} Saved")

            # Check if there is a next page
            next_button = driver.find_element(By.ID, 'lvDocuments_next')
            if "disabled" in next_button.get_attribute("class"):
                break  # Exit loop if next button is disabled

            next_button.click()  # Go to the next page
            page += 1
            random_sleep(1, 2)  # Wait for the next page to load

    except Exception as e:
        print(f"Error during scraping results: {e}")
        debug_page("scrape_results_error", filings)
    pbar.close()
    print("\n")
    return pd.DataFrame(results)


In [17]:
# old version
filings = [
    "Annual Financial Information and Operating Data",
    "Audited Financial Statements or ACFR",
    "Budget",
    "Change in Accounting Standard",
    "Change in Fiscal Year / Timing of Annual Disclosure",
    "Consultant Reports",
    "Failure to Provide Annual Financial Information as Required",
    "Information Provided to Rating Agency, Credit / Liquidity Provider or Other Third Party",
    "Interim / Additional Financial Information / Operating Data",
    "Investment / Debt / Financial Policy",
    "Other Financial / Operating Data",
    "Quarterly / Monthly Financial Information"
]

# input date range and filings
start_date = datetime(2021, 5, 5)
end_date = datetime(2021, 5, 10)
range = 10 # zero based, 1 means 2 days
filing = filings[0] # select one filing


# run the scraper
df_list = []
df_error = []
start = start_date
end = end_date
while start_date <= end_date:
    driver = setup_driver()
    mid_date = start_date + timedelta(days=range) if start_date + timedelta(days=range) < end_date else end_date
    df = perform_search(start_date, mid_date, filing)
    if df is not None:
        df_list.append(df)
    else:
        df_error.append(f"{filing}_{start_date.strftime('%Y-%m-%d')}_{mid_date.strftime('%Y-%m-%d')}")
        print(f"No results found from {start_date.strftime('%Y-%m-%d')} to {mid_date.strftime('%Y-%m-%d')}")
    
    start_date = mid_date + timedelta(days=1)
    driver.quit()
if df_list and all(df is not None for df in df_list):
    df = pd.concat(df_list)
    # df.to_csv(f"results/{filing}_combined.csv", index=False)
    print('='*50)
    print('Scraping Completed')
    print('Total length: ', len(df))
if df_error:
    print('Error dates found: ', len(df_error))
    df_error = pd.DataFrame(df_error)
    df_error.to_csv(f"results/{filing}_error_dates.csv", index=False)
    print('Saved to: ', f"results/{filing}_error_dates_{start.strftime('%Y-%m-%d')}_{end.strftime('%Y-%m-%d')}.csv")


scraping Annual Financial Information and Operating Data from 2021-05-05 to 2021-05-10
clicked on disclosures tab
No results found from 2021-05-05 to 2021-05-10
Error dates found:  1
Saved to:  results/Annual Financial Information and Operating Data_error_dates_2021-05-05_2021-05-10.csv


In [40]:
# New Version
filings = [
    "Annual Financial Information and Operating Data",
    "Audited Financial Statements or ACFR",
    "Budget",
    "Change in Accounting Standard",
    "Change in Fiscal Year / Timing of Annual Disclosure",
    "Consultant Reports",
    "Failure to Provide Annual Financial Information as Required",
    "Information Provided to Rating Agency, Credit / Liquidity Provider or Other Third Party",
    "Interim / Additional Financial Information / Operating Data",
    "Investment / Debt / Financial Policy",
    "Other Financial / Operating Data",
    "Quarterly / Monthly Financial Information"
]

# Function to clear the search
def clear_search():
    try:
        disclosures_tab = WebDriverWait(driver, 5).until(
                EC.element_to_be_clickable((By.ID, 'disclosuresFilterLi'))
            )
        disclosures_tab.click()
        disclosures_tab.click()

        clear_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.ID, 'clearSearchButton'))
        )
        clear_button.click()
        random_sleep(1, 2)  # Wait a bit to ensure the page loads completely
        print("Cleared the search")
    except Exception as e:
        print(f"Unable to clear the search: {e}")
        debug_page("clear_search_error")

# Setup driver once outside the loop
driver = setup_driver()

# input date range and filings
start_date = datetime(2021, 2, 1)
end_date = datetime(2021, 2, 10)
range = 10 # zero based, 1 means 2 days
filing = filings[0] # select one filing

# run the scraper
start = start_date
end = end_date
df_list = []
df_error = []
start = start_date
while start_date <= end_date:
    mid_date = start_date + timedelta(days=range) if start_date + timedelta(days=range) < end_date else end_date
    df = perform_search(start_date, mid_date, filing)
    if df is not None:
        df_list.append(df)
    else:
        df_error.append(f"{filing}_{start_date.strftime('%Y-%m-%d')}_{mid_date.strftime('%Y-%m-%d')}")
        print(f"No results found from {start_date.strftime('%Y-%m-%d')} to {mid_date.strftime('%Y-%m-%d')}")
    
    start_date = mid_date + timedelta(days=1)
    clear_search()  # Clear the search for the next iteration

# Quit the driver after the loop
driver.quit()

if df_list and all(df is not None for df in df_list):
    df = pd.concat(df_list)
    # df.to_csv(f"results/{filing}_combined.csv", index=False)
    print('='*50)
    print('Scraping Completed')
    print('Total length: ', len(df))
if df_error:
    print('Error dates found: ', len(df_error))
    df_error = pd.DataFrame(df_error)
    df_error.to_csv(f"results/{filing}_error_dates_{start.strftime('%Y-%m-%d')}_{end.strftime('%Y-%m-%d')}.csv", index=False)
    print('Saved to: ', f"results/{filing}_error_dates_{start.strftime('%Y-%m-%d')}_{end.strftime('%Y-%m-%d')}.csv")


scraping Annual Financial Information and Operating Data from 2021-02-01 to 2021-02-10
clicked on disclosures tab
*** page loaded
Large search result detected
Applying filter: GO
Window loaded
Results for filter GO loaded
Number of disclosures: 625


Scraping results:   0%|          | 0/625 [00:00<?, ?rows/s]

scraping page:  1


Scraping results:  16%|█▌        | 100/625 [00:02<00:11, 46.45rows/s]

scraping page:  2


...



number of rows: 732
Applying filter: DB
Window loaded
Results for filter DB loaded
Number of disclosures: 39


Scraping results:   0%|          | 0/39 [00:00<?, ?rows/s]

scraping page:  1


Scraping results: 100%|██████████| 39/39 [00:00<00:00, 40.70rows/s]




number of rows: 39
********** results/Annual Financial Information and Operating Data/results_2021-02-01_2021-02-10.csv Saved
Cleared the search
Scraping Completed
Total length:  1396


## Scraping detailed page


In [8]:
def read_csv(file_path):
    return pd.read_csv(file_path)

def download_pdf(url, file_name):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
        'Accept': 'application/pdf',
        'Referer': 'https://emma.msrb.org/' 
    }
    
    try:
        response = requests.get(url, headers=headers, stream=True)
        response.raise_for_status() 
        if not os.path.exists('downloaded_pdfs'):
            os.makedirs('downloaded_pdfs')
        download_dir = os.path.join('downloaded_pdfs', file_name)
        with open(download_dir, 'wb') as file:
            for chunk in response.iter_content(chunk_size=8192):
                file.write(chunk)
        print(f"Downloaded {file_name} successfully.")
    except requests.exceptions.RequestException as e:
        print(f"Failed to download {file_name}. Error: {e}")
    except IOError as e:
        print(f"Failed to save {file_name}. Error: {e}")
        
# get cusip text
def get_cusip_text(img_url):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
        'Accept': 'application/pdf',
        'Referer': 'https://emma.msrb.org/' 
    }
    img_response = requests.get(img_url, headers=headers, stream=True)
    img = Image.open(BytesIO(img_response.content))
    cusip_text = pytesseract.image_to_string(img, config='--psm 7').strip()
    return cusip_text

# Scrape detail page
def scrape_detail_page(link):
    try:
        driver.get(link)
        # Handle the terms of use if it appears
        if len(driver.find_elements(By.CLASS_NAME, 'contentAreaDisclaimer')) > 0:
            # print('terms_of_use found')
            handle_terms_of_use()
        # print('term_of_use handled')
        
        # Wait for the detail page to load
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, 'card-body'))
        )
        # print('find body')
        # click on the page to clear instructions
        body = driver.find_element(By.TAG_NAME, 'body')
        ActionChains(driver).move_to_element(body).click().perform()
        # print('clicked')
        random_sleep(2, 4)
        # Scrape the required information
        
        '''
        Part one
        '''
        # Extract Filing Type
    
        filing_type = WebDriverWait(driver, 5).until(
            EC.visibility_of_element_located((By.ID, 'discType'))
        ).text.strip()
        
        # Extract Disclosure Details
        disclosure_details_div = WebDriverWait(driver, 5).until(
            EC.visibility_of_element_located((By.ID, 'leftBlueBoxDiv'))
        )   
        disclosure_details = disclosure_details_div.text
        
        # Extract Contact Information
        contact_info_div = WebDriverWait(driver, 5).until(
            EC.visibility_of_element_located((By.ID, 'rightBlueBoxDiv'))
        )   
        contact_info = contact_info_div.text
        # print('part one done')
        '''
        Part two
        '''
        # Extract Document Link
        document_links = []
        document_names = []
        
        document_button = driver.find_element(By.XPATH, "//a[@id='viewDoc']")

        # Check if multiple documents are available
        if document_button.get_attribute('href'):
            document_link = document_button.get_attribute('href')
            document_name = document_link.split('/')[-1]
            document_links.append(document_link)
            document_names.append(document_name)
            # download pdf
            download_pdf(document_link, document_name)
        else:
            # Get the help attribute which contains the links
            help_attribute = document_button.get_attribute('help')
            # Parse the help attribute as HTML
            soup = BeautifulSoup(help_attribute, 'html.parser')
            # Find all the links in the help attribute
            links = [a['href'] for a in soup.find_all('a', href=True)]
            # Download each link
            for link in links:
                document_name = link.split('/')[-1]
                document_link = f"https://emma.msrb.org{link}"
                document_links.append(document_link)
                document_names.append(document_name)
                # download pdf
                download_pdf(document_link, document_name)
        # print('download_pdf done')
        
        # Click the plus button to reveal the CUSIP image
        WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//td[@class=' w30']/img[@class='detailsImg']"))
        )
        plus_button = driver.find_element(By.XPATH, "//td[@class=' w30']/img[@class='detailsImg']")
        plus_button.click()
        
        # Wait for the CUSIP image to be visible
        WebDriverWait(driver, 10).until(
            EC.visibility_of_element_located((By.XPATH, "//td/a/img[@data-cusip]"))
        )
        
        # print('cusip image found')
        # Extract and recognize text from CUSIP image
        cusip_img = driver.find_element(By.XPATH, "//td/a/img[@data-cusip]")
        img_url = cusip_img.get_attribute('src')
        cusip_text = get_cusip_text(img_url)

        # print(cusip_text)
        return {
            'filing_type': filing_type,
            'disclosure_details': disclosure_details,
            'contact_info': contact_info,
            'document_links': document_links,
            'document_names': document_names,
            'cusip_text': cusip_text
        }

    except Exception as e:
        print(f"Error scraping detail page {link}: {e}")
        debug_page(f"{link.split('/')[-1]}_error")
        return None


def scrape_detail_pages_from_csv(file_path):
    df = read_csv(file_path)
    detail_data = []
    failed_links = []
    # set up tqdm progress bar
    pbar = tqdm(total=len(df), desc='Scraping detail pages', unit='rows', dynamic_ncols=True)
    
    for index, row in df.iterrows():
        link = row['link']
        print('='*50)
        print('Sraping: ',link)
        detail_info = scrape_detail_page(link)
        if detail_info:
            detail_data.append(detail_info)
        else:
            failed_links.append(link)
        # save temp data
        detail_df = pd.DataFrame(detail_data)
        detail_df.to_csv(f'temp_detail_data_{file_path}', index=False)
        failed_links_df = pd.DataFrame(failed_links)
        failed_links_df.to_csv(f'temp_failed_links_{file_path}', index=False)
        # Optional: Add a delay to mimic human behavior and avoid detection
        random_sleep(2, 4)
        pbar.update(1)
        
    pbar.close()
    driver.quit()
    detail_df = pd.DataFrame(detail_data)
    print(f"Failed to scrape {len(failed_links)} links: {failed_links}")

    return detail_df


In [12]:
# csv file
# file_path = 'results_2024-04-24_2024-04-24.csv'

file_path = 'results/Annual Financial Information and Operating Data/results_2021-01-01_2021-01-02.csv'

# Initialize the WebDriver
driver = setup_driver()

# Use the function to scrape details from the CSV
detail_df = scrape_detail_pages_from_csv(file_path)
detail_df.to_csv(f'detail_data_{file_path}', index=False)
print(detail_df.head())

Sraping:  https://emma.msrb.org/MarketActivity/ContinuingDisclosureDetails/P21089859


KeyboardInterrupt: 